In [ ]:
import numpy as np
import cv2
import time
from IPython.display import clear_output

def get_available_cameras (upper_bound = 10, lower_bound = 0):
    available = []
    
    for i in range (lower_bound, upper_bound):
        cap = cv2.VideoCapture (i)
    
        if (cap.isOpened ()):
            available.append (i)
    
        cap.release ()
    
    return available

def put_text (img, text, str_num):
    font                   = cv2.FONT_ITALIC
    bottomLeftCornerOfText = (10, 50 + 30 * str_num)
    fontScale              = 0.8
    fontColor              = (15, 15, 215)
    lineType               = 2
    
    cv2.putText(img, text, 
        bottomLeftCornerOfText, 
        font, 
        fontScale,
        fontColor,
        lineType)

CAMERA = 0
VIDEO  = 1
PHOTO  = 2

video_path = ""
video_file = ""

#photo_path = "/Users/elijah/Dropbox/Programming/RoboCup/nao_cv/floor_desk/"
#photo_file = "IMAGE_0_0_35.jpg"

photo_path = "/Users/elijah/Desktop/"
photo_file = "Снимок экрана 2019-05-15 в 18.00.08.png"

output_path = "/Users/elijah/Dropbox/Programming/RoboCup/nao_cv/geometrical/chessboard_images/"

class filter_:
    def __init__ (self, is_mask_ = False):
        self.is_mask = is_mask_
        pass
    
    def apply_filter (self, frame, iter_num):
        return frame

class moving_rectangular_mask (filter_):
    def __init__ (self, vertical_, thickness_, speed_, start_ = 0):
        filter_.__init__ (self, is_mask_ = True)
        
        self.coord     = start_
        self.vertical  = vertical_
        self.thickness = thickness_
        self.speed     = speed_
    
    def apply_filter (self, frame, iter_num):
        sh = frame.shape
        
        mask = np.zeros_like (frame, np.float64)
        
        self.coord += self.speed
        
        if (self.vertical == True):
            if (self.coord >= sh [0]):
                self.coord = self.coord % sh [0]
            
            mask [self.coord : min (self.coord + self.thickness, sh [0]), :, :] = 1
            
            if (self.coord + self.thickness >= sh [0]):
                mask [0 : self.coord + self.thickness - sh [0], :] = 1

        else:
            if (self.coord >= sh [1]):
                self.coord = self.coord % sh [1]
            
            mask [:, self.coord : min (self.coord + self.thickness, sh [1]), :] = 1
            
            if (self.coord + self.thickness >= sh [1]):
                mask [:, 0 : self.coord + self.thickness - sh [1], :] = 1
        
        return mask

class invert (filter_):
    def __init__ (self):
        filter_.__init__ (self)
        pass
    
    def apply_filter (self, frame, iter_num, mask = None):
        inverted = 255 - frame
        
        if (mask is None):
            result = inverted
        
        else:
            inv = cv2.bitwise_not (frame, mask = cv2.inRange (mask, (0.9, 0.9, 0.9), (1.1, 1.1, 1.1)))
            rest = cv2.bitwise_and (frame, frame, mask = cv2.inRange (mask, (-0.9, -0.9, -0.9), (0.99, 0.99, 0.99)))
            result = np.add (inv, rest)
            
        return result

class intensity (filter_):
    def __init__ (self, coeff_):
        filter_.__init__ (self)
        
        self.coeff = coeff_
        
        pass
    
    def apply_filter (self, frame, iter_num, mask = None):
        changed = np.array (np.multiply (frame, self.coeff), np.uint8)
        
        if (mask is None):
            result = changed
        
        else:
            cut  = cv2.bitwise_and (changed, changed, mask = cv2.inRange (mask, (0.9, 0.9, 0.9), (1.1, 1.1, 1.1)))
            rest = cv2.bitwise_and (frame, frame, mask = cv2.inRange (mask, (-0.9, -0.9, -0.9), (0.99, 0.99, 0.99)))
            
            result = np.add (cut, rest)
            
        return result

class noise (filter_):
    def __init__ (self, scale_):
        filter_.__init__ (self)
        
        self.scale = scale_
        
        pass
    
    def apply_filter (self, frame, iter_num, mask = None):
        sh = frame.shape
        
        noised = np.add (np.array (np.multiply (np.random.randn (sh [0], sh [1], sh [2]), self.scale), np.uint8), frame)
        
        if (mask is None):
            result = noised
        
        else:
            cut  = cv2.bitwise_and (noised, noised, mask = cv2.inRange (mask, (0.9, 0.9, 0.9), (1.1, 1.1, 1.1)))
            rest = cv2.bitwise_and (frame, frame, mask = cv2.inRange (mask, (-0.9, -0.9, -0.9), (0.99, 0.99, 0.99)))
            
            result = noised
            
            result = np.add (cut, rest)
            
        return result

class translate_channel (filter_):
    def __init__ (self, channel_num_, shift_x_, shift_y_):
        filter_.__init__ (self)
        
        self.channel_num = channel_num_
        self.shift_x = shift_x_
        self.shift_y = shift_y_
    
    def apply_filter (self, frame, iter_num):
        sh = frame.shape
        szx = sh [0]
        szy = sh [1]
        
        frame [max (0, self.shift_x) : min (szx, szx + self.shift_x),
               max (0, self.shift_y) : min (szy, szy + self.shift_y),
               self.channel_num] = \
        frame [max (0, - self.shift_x) : min (szx, szx - self.shift_x),
               max (0, - self.shift_y) : min (szy, szy - self.shift_y),
               self.channel_num]
        
        return frame
    
def apply_filters (frame, filters, iter_num):
    for i in range (len (filters)):
        if (filters [i].is_mask == True):
            continue
        
        if (i - 1 >= 0 and filters [i - 1].is_mask == True):
            frame = filters [i].apply_filter (frame, iter_num, filters [i - 1].apply_filter (frame, iter_num))
        
        else:
            frame = filters [i].apply_filter (frame, iter_num)
        
    return frame

def main ():
    INPUT_SOURCE = CAMERA
    cam_num = min (get_available_cameras ())
    cam = cv2.VideoCapture (cam_num)

    if (INPUT_SOURCE != CAMERA):
        cam.release ()

    if (INPUT_SOURCE == VIDEO):
        cam = cv2.VideoCapture (video_path + video_file)

    elif (INPUT_SOURCE == PHOTO):
        img = cv2.imread (photo_path + photo_file)

    cv2.namedWindow ("frame", cv2.WINDOW_NORMAL)
    cv2.resizeWindow ("frame", (1280, 720))
    
    iter_num = 0
    
    filters = []
    
    filters.append (translate_channel (1, 15, 10))
    filters.append (translate_channel (0, 5, -10))
    
    filters.append (moving_rectangular_mask (True, 90, 6, 50))
    filters.append (intensity (0.55))

    #filters.append (moving_rectangular_mask (True, 100, -7, 450))
    #filters.append (noise (8))

    #filters.append (moving_rectangular_mask (True, 90, 5, 150))
    #filters.append (noise (55))

    filters.append (moving_rectangular_mask (True, 90, 9))
    filters.append (invert ())

    #filters.append (moving_rectangular_mask (False, 6, 36))
    #filters.append (invert ())

    filters.append (moving_rectangular_mask (True, 9, 26))
    filters.append (invert ())

    #filters.append (moving_rectangular_mask (False, 40, 17))
    #filters.append (invert ())
    
    while (True):
        if (INPUT_SOURCE == CAMERA or INPUT_SOURCE == VIDEO):
            ret, frame_ = cam.read ()

        elif (INPUT_SOURCE == PHOTO):
            frame_ = img.copy ()

        #cv2.waitKey (1)    
        
        frame = apply_filters (frame_, filters, iter_num)
        
        cv2.imshow ("frame", frame)

        #time.sleep (0.01)

        clear_output (wait=True)
        
        keyb = cv2.waitKey (1) & 0xFF
        
        iter_num += 1
        
        if (keyb == ord('q')):
            break

    cam.release ()

    cv2.destroyAllWindows()

if __name__ == "__main__":
    main ()

In [ ]:
https://photoshop-master.ru/lessons/effects/sozdayom_effekt_pomeh_vhs_v_fotoshop.html